In [13]:
import re
import pandas as pd
from decimal import Decimal

# Define regular expressions
name_re = r'([A-Za-z_0-9]+)(\[.*)?:'
support_re = r'support=([0-9.]+)'
alt_re = r'alternativePlacements=({.*?})'
mut_re = r'mutations=({.*?})'
ns_re = r'Ns=({.*?})'
len_re = r':([0-9.e-]+)'

columns = ['name', 'support', 'alternativePlacements', 'mutations', 'Ns', 'length']

with open('/homes/zihao/EBI_INTER/A_Datas/MAPLE0.3.2_nexusTree_1000samples_simulations_repl1.tree') as f:
    for line in f:
        if line.strip() == 'begin trees;':
            break

    # 读取数据行
    data = ''
    for line in f:
        if line.strip() == ';':
            break
        data += line

# 将数据分割为多行
data = re.sub(r'\s+', '', data)
data = data.split(';')[0] + ';'

# 匹配所有的节点信息并存入列表
node_list = re.findall(r'([A-Za-z_0-9]+\[.*?\]):([0-9.e-]+)', data)

# 将节点信息列表转为DataFrame
df = pd.DataFrame(node_list, columns=['node', 'length'])

# 使用正则表达式提取节点信息的各个字段
df['name'] = df['node'].apply(lambda x: re.findall(name_re, x)[0][0])
df['support'] = df['node'].apply(lambda x: re.findall(support_re, x))
df['support'] = df['support'].apply(lambda x: x[0] if len(x) > 0 else '')
df['alternativePlacements'] = df['node'].apply(lambda x: re.findall(alt_re, x))
df['alternativePlacements'] = df['alternativePlacements'].apply(lambda x: ','.join(re.findall(r'([A-Za-z_0-9]+):', x[0])) if len(x) > 0 and '{' in x[0] else '')
df['mutations'] = df['node'].apply(lambda x: re.findall(mut_re, x))
df['mutations'] = df['mutations'].apply(lambda x: ','.join(re.findall(r'([A-Z]\d+[A-Z]):', x[0])) if len(x) > 0 and '{' in x[0] else '')
df['Ns'] = df['node'].apply(lambda x: re.findall(ns_re, x))
df['Ns'] = df['Ns'].apply(lambda x: x[0] if len(x) > 0 else '')
df['length'] = df['length'].apply(lambda x: str(Decimal(x)))

# 重新排列列的顺序
df = df[columns]
df

,name,support,alternativePlacements,mutations,Ns,length
0,29,0.8546038249510233,in1,"T2350C,G2409T,T3834C,G20083A,T29068C",{},0.0001645498048891425
1,31,1.0,,G23568T,"{1-39,29837-29879}",0.000032907579649191664
2,21,1.0,,"G4807A,G21848T","{1-54,29837-29879}",0.00006636060630025907
3,07,0.9998932453172774,,"C13501T,G19204T,T19281C,A19761T,C21207T,G23405...","{1-38,7299-7299,11288-11296,21765-21770,21991-...",0.00023486416835291933
4,24,0.9997957717116446,,"C794T,G7693T,G8872T,G9116A,C12823T,G13651T,C14...","{1-54,29768-29879}",0.00033656734387543433
...,...,...,...,...,...,...
1153,03,0.9896527437388796,in2,"G10877T,C13181T,G16897T,G18674A,G25540A","{1-21,29832-29858}",0.00015704254582052127
1154,20,0.9896252899842426,in2,"G11774T,G13771T,C25678T","{1-54,10738-10873,10888-11022,11288-11296,2176...",0.00009938909399086092
1155,in4,0.8553865516037018,in1,A16044G,"{1-21,29837-29858}",0.00003932770412099634
1156,in2,1.0,,"T8648G,A13193G,G15496A,T19730G,T24626G,C26613T...",{},0.000049395613418929594
